# Part 1 of the Assignment

Lets first load required libraries:

In [37]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium
import folium # map rendering library

print("Libraries imported.")

Libraries imported.


Scrap data from Wikipedia page into a DataFrame:

In [12]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

# create three lists to store table data
postalCodes = []
boroughs = []
neighborhoods = []

# append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodes.append(cells[0].text.rstrip('\n'))
        boroughs.append(cells[1].text.rstrip('\n'))
        neighborhoods.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell
        
len(postalCodes)


180

Create a dataframe and view the list.

In [14]:
# create a new DataFrame from the three lists
toronto_df = pd.DataFrame({"PostalCode": postalCodes,
                           "Borough": boroughs,
                           "Neighborhood": neighborhoods})

toronto_df.tail()

,PostalCode,Borough,Neighborhood
175,M5Z,Not assigned,
176,M6Z,Not assigned,
177,M7Z,Not assigned,
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
179,M9Z,Not assigned,


Drop cells with borough that is "Not assigned"

In [16]:
# drop cells with a borough that is Not assigned
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_dropna.tail()

,PostalCode,Borough,Neighborhood
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Group theneighbourhoods in the same borough

In [19]:
# group neighborhoods in the same borough
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [29]:
#check if there's a borough with no neighbourhood
emptyneighbourhood = toronto_df_grouped[toronto_df_grouped['Neighborhood'] == "Not assigned"]
emptyneighbourhood.head()

,PostalCode,Borough,Neighborhood


Replicate the dataframe provided in the assignemnt instructions

In [65]:
# filter the postal codes as captured in the assignment instructions
sample_df = toronto_df_grouped[toronto_df_grouped['PostalCode'].isin(["M5G", "M2H", "M4B", "M1J", 
                                                                      "M4G", "M4M", "M1R", "M9V", 
                                                                      "M9L", "M5V", "M1B", "M5A"]) ]
sample_df.reset_index(drop=True)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1J,Scarborough,Scarborough Village
2,M1R,Scarborough,"Wexford, Maryvale"
3,M2H,North York,Hillcrest Village
4,M4B,East York,"Parkview Hill, Woodbine Gardens"
5,M4G,East York,Leaside
6,M4M,East Toronto,Studio District
7,M5A,Downtown Toronto,"Regent Park, Harbourfront"
8,M5G,Downtown Toronto,Central Bay Street
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [33]:
# print the number of rows of the final dataframe
toronto_df_grouped.shape

(103, 3)

# Part 2 of the Assignment

In [52]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_d98731a308274755a79c8609535584b1 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Bmgb9nmiNNZhBcQhU3e9vKB9ZIOs-yPyLHmMjMesTYzx',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_d98731a308274755a79c8609535584b1.get_object(Bucket='machinelearningwithpythoncoursera-donotdelete-pr-m16tkampsetrhx',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

In [53]:
# body object from IBM reference to the CSV file
coordinates = pd.read_csv(body)
coordinates.head()


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge the two files

In [54]:
# merge two table on the column "PostalCode"
new_toronto_df = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
new_toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [64]:
# filter the postal codes as captured in the assignment instructions
new_sample_df = new_toronto_df[new_toronto_df['PostalCode'].isin(["M5G", "M2H", "M4B", 
                                                                  "M1J", "M4G", "M4M", 
                                                                  "M1R", "M9V", "M9L", 
                                                                  "M5V", "M1B", "M5A"]) ]
new_sample_df.reset_index(drop=True)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
2,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
3,M2H,North York,Hillcrest Village,43.803762,-79.363452
4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
5,M4G,East York,Leaside,43.709060,-79.363452
6,M4M,East Toronto,Studio District,43.659526,-79.340923
7,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
8,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


# Part 3 of the Assignment

Use geopy library to get the latitude and longitude values of Toronto

In [56]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="my-toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Create a map of Toronto

In [57]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(new_sample_df['Latitude'], new_sample_df['Longitude'], 
                                           new_sample_df['Borough'], new_sample_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

If we work with neighbourhoods with only the words Toronto

In [66]:
toronto_data = new_sample_df[new_sample_df['Borough'].str.contains("oronto")].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4M,East Toronto,Studio District,43.659526,-79.340923
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
2,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
3,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


In [67]:

# create map of Toronto using latitude and longitude values
map_toronto2 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], 
                                           toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto2)  
    
map_toronto2

## Use the Foursquare API to explore the neighborhoods

In [68]:
# @hidden_cell
CLIENT_ID = 'AJZAXQMDFWSCS5GOMJ45ECTGALBIXVIDZ5A0UA2RVQWXOC0F' # your Foursquare ID
CLIENT_SECRET = 'LJ52P52IVZ1O35LHYOC4AXD4VP00IC1BQU4TDI0VJBBNJU1Q' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Let's get the top 100 venues that are within a radius of 500 meters.

In [69]:
radius = 500
LIMIT = 100
venues = []

for lat, long, post, borough, neighborhood in zip(new_sample_df['Latitude'], new_sample_df['Longitude'], 
                                                  new_sample_df['PostalCode'], new_sample_df['Borough'], 
                                                  new_sample_df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID, CLIENT_SECRET, VERSION, lat, long, radius, LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
        
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 
                     'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(233, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,McCowan Park,43.745089,-79.239336,Playground
2,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,Primo Pizza,43.744759,-79.244751,Pizza Place
3,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849,Crown Pastries,43.746098,-79.293142,Bakery
4,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849,Wexford Restaurant,43.746030,-79.293843,Breakfast Spot


How many venues for each neighbourhood?

In [72]:
venues_df.groupby(["Neighborhood"]).count()

,PostalCode,Borough,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,,,
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15,15,15
Central Bay Street,63,63,63,63,63,63,63,63
Hillcrest Village,5,5,5,5,5,5,5,5
Humber Summit,2,2,2,2,2,2,2,2
Leaside,32,32,32,32,32,32,32,32
"Malvern, Rouge",1,1,1,1,1,1,1,1
"Parkview Hill, Woodbine Gardens",11,11,11,11,11,11,11,11
"Regent Park, Harbourfront",45,45,45,45,45,45,45,45
Scarborough Village,2,2,2,2,2,2,2,2


How many unique categories?

In [76]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 111 uniques categories.


## Analyze each neighbourhood

In [99]:

# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
#toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
#toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-1:]) + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(233, 112)


,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Athletics & Sports,Auto Garage,Bagel Shop,Bakery,Bank,Bar,Beer Store,Bike Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Café,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Gay Bar,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Liquor Store,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Neighborhood,Office,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Poke Place,Pool,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Salad Place,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoothie Shop,Spa,Sporting Goods Shop,Sports Bar,Stationery Store,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Scarborough Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Scarborough Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Wexford, Maryvale",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Wexford, Maryvale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [102]:
toronto_onehot.shape

(233, 112)

Now let's group rows by neighbourhood  

In [103]:
toronto_grouped = toronto_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped.head()

(12, 112)


,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Athletics & Sports,Auto Garage,Bagel Shop,Bakery,Bank,Bar,Beer Store,Bike Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Café,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Gay Bar,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Liquor Store,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Neighborhood,Office,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Poke Place,Pool,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Salad Place,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoothie Shop,Spa,Sporting Goods Shop,Sports Bar,Stationery Store,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"CN Tower, King and Spadina, Railway Lands, Har...",0.066667,0.066667,0.066667,0.066667,0.133333,0.066667,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0000,0.066667,0.00000,0.00000,0.066667,0.0,0.066667,0.00000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.066667,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0000,0.000000,0.0,0.0,0.00000,0.00000,0.000000,0.066667,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.066667,0.0,0.000000,0.0,0.0,0.000000,0.066667,0.000000,0.000000,0.000000,0.066667,0.000000,0.0,0.00000,0.00000,0.000000,0.0000,0.00000,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
1,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.015873,0.0,0.0,0.00000,0.0,0.0000,0.015873,0.00000,0.00000,0.000000,0.0,0.000000,0.00000,0.00000,0.031746,0.031746,0.063492,0.0,0.015873,0.0,0.0,0.174603,0.0,0.015873,0.0,0.0,0.0,0.031746,0.015873,0.015873,0.015873,0.0,0.0,0.015873,0.00000,0.0,0.015873,0.0,0.0,0.00000,0.0,0.015873,0.0,0.0000,0.015873,0.0,0.0,0.00000,0.00000,0.015873,0.000000,0.0,0.0,0.0,0.015873,0.015873,0.015873,0.0,0.063492,0.031746,0.015873,0.0,0.00000,0.0,0.00000,0.015873,0.015873,0.0,0.015873,0.015873,0.0,0.00000,0.0,0.0,0.000000,0.0,0.015873,0.0,0.0,0.015873,0.000000,0.015873,0.031746,0.047619,0.000000,0.015873,0.0,0.00000,0.00000,0.015873,0.0000,0.00000,0.0,0.00000,0.015873,0.015873,0.0,0.015873,0.015873,0.0,0.015873,0.015873
2,Hillcrest Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0000,0.000000,0.00000,0.00000,0.000000,0.0,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.2,0.000000,0.00000,0.0,0.000000,0.0,0.2,0.00000,0.0,0.000000,0.0,0.0000,0.000000,0.0,0.2,0.00000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.2,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.00000,0.000000,0.0000,0.00000,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.

Top 5 venues for each postcode

In [106]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhoods'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0     Airport Service  0.13
1             Airport  0.07
2                 Bar  0.07
3  Airport Food Court  0.07
4               Plane  0.07


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.17
1                 Café  0.06
2   Italian Restaurant  0.06
3       Sandwich Place  0.05
4  Japanese Restaurant  0.03


----Hillcrest Village----
                      venue  freq
0               Golf Course   0.2
1                   Dog Run   0.2
2                      Pool   0.2
3      Fast Food Restaurant   0.2
4  Mediterranean Restaurant   0.2


----Humber Summit----
                       venue  freq
0                Pizza Place   0.5
1               Home Service   0.5
2  Latin American Restaurant   0.0
3                   Pharmacy   0.0
4                  Pet Store   0.0


----Leaside----
                    venue  fre

Let's write a function to sort the venues in descending order. 
Then create the new dataframe and display the top 10 venues for each neighborhood

In [107]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [155]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport,Harbor / Marina,Plane,Boutique,Rental Car Location,Boat or Ferry,Coffee Shop,Bar,Sculpture Garden
1,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Bubble Tea Shop,Burger Joint,Department Store,Japanese Restaurant,Salad Place,Yoga Studio
2,Hillcrest Village,Golf Course,Pool,Fast Food Restaurant,Mediterranean Restaurant,Dog Run,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop
3,Humber Summit,Pizza Place,Home Service,Discount Store,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop
4,Leaside,Coffee Shop,Sporting Goods Shop,Furniture / Home Store,Bank,Burger Joint,Breakfast Spot,Beer Store,Bike Shop,Pet Store,Brewery


## Cluster the areas

In [156]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 3, 1, 0, 1, 1, 2, 1], dtype=int32)

In [157]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
#new_sample_df
toronto_merged = new_sample_df  

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhoods'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0,Fast Food Restaurant,Yoga Studio,Distribution Center,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,2,Pizza Place,Playground,Cheese Shop,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop
11,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849,1,Bakery,Vietnamese Restaurant,Shopping Mall,Middle Eastern Restaurant,Auto Garage,Breakfast Spot,Sandwich Place,Department Store,Diner,Dessert Shop
17,M2H,North York,Hillcrest Village,43.803762,-79.363452,1,Golf Course,Pool,Fast Food Restaurant,Mediterranean Restaurant,Dog Run,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop
35,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1,Pizza Place,Pharmacy,Athletics & Sports,Fast Food Restaurant,Gym / Fitness Center,Intersection,Pet Store,Breakfast Spot,Bank,Gastropub


## visualize with a map

In [158]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], 
                                  toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## examine the four clusters

Cluster 1

In [168]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Malvern, Rouge",Fast Food Restaurant,Yoga Studio,Distribution Center,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space


Cluster 2

In [169]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,"Wexford, Maryvale",Bakery,Vietnamese Restaurant,Shopping Mall,Middle Eastern Restaurant,Auto Garage,Breakfast Spot,Sandwich Place,Department Store,Diner,Dessert Shop
17,Hillcrest Village,Golf Course,Pool,Fast Food Restaurant,Mediterranean Restaurant,Dog Run,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop
35,"Parkview Hill, Woodbine Gardens",Pizza Place,Pharmacy,Athletics & Sports,Fast Food Restaurant,Gym / Fitness Center,Intersection,Pet Store,Breakfast Spot,Bank,Gastropub
38,Leaside,Coffee Shop,Sporting Goods Shop,Furniture / Home Store,Bank,Burger Joint,Breakfast Spot,Beer Store,Bike Shop,Pet Store,Brewery
43,Studio District,Café,Coffee Shop,American Restaurant,Bakery,Brewery,Gastropub,Wine Bar,Pet Store,Park,Cheese Shop
53,"Regent Park, Harbourfront",Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Ice Cream Shop,Chocolate Shop,Mexican Restaurant
57,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Bubble Tea Shop,Burger Joint,Department Store,Japanese Restaurant,Salad Place,Yoga Studio
68,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport,Harbor / Marina,Plane,Boutique,Rental Car Location,Boat or Ferry,Coffee Shop,Bar,Sculpture Garden
101,"South Steeles, Silverstone, Humbergate, Jamest...",Grocery Store,Pharmacy,Beer Store,Sandwich Place,Liquor Store,Japanese Restaurant,Fast Food Restaurant,Pizza Place,Fried Chicken Joint,Clothing Store


CLuster 3

In [170]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough Village,Pizza Place,Playground,Cheese Shop,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop


Cluster 4

In [171]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
96,Humber Summit,Pizza Place,Home Service,Discount Store,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop


## Observation

Most of the neighbourhoods have common attributes, fall in a busy commecial region (cluster 2). Malvern, Rouge (cluster 1) looks like  suburb,  in the outskirsts. Scarborough Village (cluster 3)  & Humber Summit (cluster 4) are eatery places.